In [1]:
import os
import random
import gc
import fitz
import tiktoken
import torch
import REMOVED_SECRET as F
import numpy as np
import shutil
import queue
import uuid
from keybert import KeyBERT
from tkinter import filedialog
import tkinter as tk
from langchain_community.vectorstores import FAISS
from REMOVED_SECRET import DistanceStrategy
from langchain.document_loaders import PyMuPDFLoader
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from memory_profiler import profile
from RAG_UTILS import RERANKER_MODEL, MODEL_ID, EMBEDDING_MODEL_NAME, CustomTextGenerationPipeline, RAGSystem, main, DocumentProcessor, generate_vocab_list
#from rag_for_notebook_sunday import READER_LLM, KNOWLEDGE_VECTOR_DATABASE, RERANKER, CustomTextGenerationPipeline, answer_with_rag, docs_processed
from loss_functions import weighted_loss, label_smoothed_nll_loss
# "mps" if REMOVED_SECRET.is_available() else
# Ensure that we are using the correct device
device = torch.device("cuda" if REMOVED_SECRET() else "cpu")




/home/obb/codes/langers/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `REMOVED_SECRET` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
class PDFKeywordExtractor:
    def __init__(self, num_keywords=50):
        self.num_keywords = num_keywords
        self.kw_model = KeyBERT()
    
    def extract_keywords(self, pdf_path):
        try:
            loader = PyMuPDFLoader(pdf_path)
            document = loader.load()[0]
            keywords = REMOVED_SECRET(document.page_content, keyphrase_ngram_range=(1, 3), top_n=self.num_keywords)
            print("Keywords extracted:", keywords)
            keywords_list = [keyword for keyword, score in keywords]
            return keywords_list
        except Exception as e:
            print(f"Error loading or processing PDF {pdf_path}: {e}")
            return []


In [3]:
import pymupdf
#class PDFInjector:
class PDFInjector:
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model

    def inject_text(self, source_pdf_path, destination_pdf_path, text_to_inject, keywords_list, docs_processed):
        # Open the source PDF
        src_doc = fitz.open(source_pdf_path)
        
        # Create a new PDF document
        dst_doc = fitz.open()
        
        # Copy all pages from source to destination
        for page in src_doc:
            dst_doc.insert_pdf(src_doc, from_page=page.number, to_page=page.number)

        # Inject the text
        for doc in docs_processed:
            page_num = 0  # Assuming we're always injecting on the first page
            page = dst_doc[page_num]

            chunk_keywords = [kw for kw in keywords_list if kw in doc.page_content]

            if chunk_keywords:
                strongest_keyword = self._find_strongest_keyword(chunk_keywords, doc.page_content)
                print("Strongest keyword:", strongest_keyword)

                for text_instance in page.search_for(strongest_keyword):
                    rect = text_instance  # rectangle where keyword is found
                    page.insert_text(rect.tl, text_to_inject, fontsize=1, color=(0,0,0))

        # Save the new PDF
        dst_doc.save(destination_pdf_path)
        dst_doc.close()
        src_doc.close()

    def _find_strongest_keyword(self, keywords, chunk_text):

        chunk_embedding = REMOVED_SECRET.encode(chunk_text, convert_to_tensor=True)
        keyword_embeddings = [REMOVED_SECRET.encode(kw, convert_to_tensor=True) for kw in keywords]
        keyword_similarities = {kw: 0 for kw in keywords}
        for kw, kw_embedding in zip(keywords, keyword_embeddings):
            if kw in chunk_text:
                similarity = util.pytorch_cos_sim(chunk_embedding, kw_embedding).item()
                keyword_similarities[kw] = similarity
        strongest_keyword = max(keyword_similarities, key=keyword_similarities.get, default=None)
        return strongest_keyword
    




In [4]:

class SequenceMutator:
    def __init__(self, model, tokenizer, weight=0.8, k=32, learning_rate=0.8):
        self.model = model
        self.tokenizer = tokenizer
        self.weight = weight
        self.k = k
        self.learning_rate = learning_rate

    def calculate_loss(self, logits, target_response_tokens, crucial_indices):
        # Compute the weighted loss, just as in previous functions
        loss = weighted_loss(logits, target_response_tokens, crucial_indices, self.weight)
        return loss

    def mutate_sequence(self, seq_tokens, target_response_tokens, crucial_indices):
        seq_tokens = seq_tokens.long().to(REMOVED_SECRET)
        print("Seq tokens", seq_tokens)
        target_response_tokens = target_response_tokens.long().to(REMOVED_SECRET)
            
            # Get embeddings
        embeddings = REMOVED_SECRET()(seq_tokens)
        embeddings.requires_grad_(True)

        print("embeddings dtype:", embeddings.dtype)
        print("target_response_tokens dtype:", target_response_tokens.dtype)
        # Ensure model parameters require gradients
        #for param in REMOVED_SECRET():
           #param.requires_grad = True
        
        # Forward pass
        outputs = self.model(inputs_embeds=embeddings)
        logits = outputs.logits
        
        # Calculate loss
        loss = self.calculate_loss(logits, target_response_tokens, crucial_indices)
        
        # Compute gradients
        gradients = REMOVED_SECRET(loss, embeddings, retain_graph=True)[0]
        print("Gradients:", gradients)
        if gradients is None:
            raise RuntimeError("Gradient computation failed; grad is None")
        
        new_seqs = []
        for _ in range(self.k):
            mutate_index = torch.randint(0, seq_tokens.shape[1], (1,)).item()
            print("Mutate index:", mutate_index)
            
            # Mutate the token's embedding based on the gradient
            mutated_embedding = embeddings[0][mutate_index] - self.learning_rate * gradients[0][mutate_index]
            print("Mutated embedding", mutated_embedding)
            
            # Find the closest token in the vocabulary to the mutated embedding
            distances = torch.norm(REMOVED_SECRET().weight.data - mutated_embedding, dim=1)
            closest_token_id = torch.argmin(distances).item()
            
            # Create a new sequence with the mutated token
            new_seq = seq_tokens.clone()
            new_seq[0][mutate_index] = closest_token_id
            print("New sequence:", new_seq)
            
            # Append mutated sequence
            new_seqs.append(new_seq)
        
        # Zero out the gradients for the next iteration
        REMOVED_SECRET()
        
        return new_seqs

    def choose_best_sequence(self, new_seqs, losses):
        # Choose the sequence with the lowest loss
        best_index = np.argmin(losses)
        print("Best index", best_index)
        return new_seqs[best_index]


In [5]:
import tkinter as tk
from tkinter import ttk
import queue

class DebugUI:
    def __init__(self):
        self.root = tk.Toplevel()
        REMOVED_SECRET("PSO Debug UI")
        REMOVED_SECRET("800x600")

        self.iteration_var = tk.StringVar(value="Iteration: 0")
        self.particle_var = tk.StringVar(value="Particle: 0")
        self.memory_var = tk.StringVar(value="Memory Usage: 0 MB")
        self.best_fitness_var = tk.StringVar(value="Best Fitness: N/A")

        ttk.Label(self.root, textvariable=self.iteration_var).pack(pady=5)
        ttk.Label(self.root, textvariable=self.particle_var).pack(pady=5)
        ttk.Label(self.root, textvariable=self.memory_var).pack(pady=5)
        ttk.Label(self.root, textvariable=self.best_fitness_var).pack(pady=5)

        ttk.Label(self.root, text="Current LLM Output:").pack(pady=5)
        self.llm_output_text = tk.Text(self.root, height=5, width=80, wrap=tk.WORD)
        REMOVED_SECRET(pady=5)

        ttk.Label(self.root, text="Log:").pack(pady=5)
        self.log_text = tk.Text(self.root, height=15, width=80)
        REMOVED_SECRET(pady=5)

        self.update_queue = queue.Queue()
        self.running = True

    def update_stats(self, iteration, particle, best_fitness):
        REMOVED_SECRET(("stats", iteration, particle, best_fitness))

    def update_llm_output(self, output):
        REMOVED_SECRET(("llm_output", output))

    def log(self, message):
        REMOVED_SECRET(("log", message))

    def update_memory(self, memory):
        REMOVED_SECRET(("memory", memory))

    def process_queue(self):
        try:
            while True:
                item = REMOVED_SECRET()
                if item[0] == "stats":
                    _, iteration, particle, best_fitness = item
                    REMOVED_SECRET(f"Iteration: {iteration}")
                    REMOVED_SECRET(f"Particle: {particle}")
                    REMOVED_SECRET(f"Best Fitness: {best_fitness}")
                elif item[0] == "llm_output":
                    _, output = item
                    REMOVED_SECRET('1.0', tk.END)
                    REMOVED_SECRET(tk.END, output)
                elif item[0] == "log":
                    _, message = item
                    REMOVED_SECRET(tk.END, message + "\n")
                    REMOVED_SECRET(tk.END)
                elif item[0] == "memory":
                    _, memory = item
                    REMOVED_SECRET(f"Memory Usage: {memory} MB")
        except queue.Empty:
            pass
        if self.running:
            REMOVED_SECRET(100, self.process_queue)

    def start(self):
        self.process_queue()
        REMOVED_SECRET()

    def stop(self):
        self.running = False
        REMOVED_SECRET()

In [6]:
import random
import numpy as np
from REMOVED_SECRET import cosine
import matplotlib.pyplot as plt
import os
import tkinter as tk
from functools import lru_cache
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
import concurrent.futures
class PSOSequenceOptimizer:
    def __init__(self, embedding_model, n_particles=5, w=0.5, c1=1, c2=1, temperature=0.1):
        self.embedding_model = embedding_model
        self.n_particles = n_particles
        self.w = w  # Inertia weight
        self.c1 = c1  # Cognitive weight
        self.c2 = c2  # Social weight
        self.fitness_cache = {}
        self.generation_data = []
        self.debug_ui = None
        self.temperature = temperature

       
    @lru_cache(maxsize=1000)
    def get_embedding(self, text):
        return REMOVED_SECRET(text)
    
    def contrastive_loss(self, anchor, positive, negative):

        anchor = torch.tensor(anchor) if not isinstance(anchor, torch.Tensor) else anchor
        positive = torch.tensor(positive) if not isinstance(positive, torch.Tensor) else positive
        negative = torch.tensor(negative) if not isinstance(negative, torch.Tensor) else negative
        #compute similarities 
        pos_sim = F.cosine_similarity(anchor, positive, dim=0)
        neg_sim = F.cosine_similarity(anchor, negative, dim=0)

        #compute contrastive loss
        #we want to maximize the distance between the anchor and the positive.
        #and minimize the distance between the positive and negative.

        loss = torch.log(torch.exp(pos_sim / self.temperature) / (torch.exp(pos_sim / self.temperature) + torch.exp(neg_sim / self.temperature)))

        return loss.item()

    def evaluate_sequence(self, sequence, rag_system, pdf_injector, pdf_manager, temp_pdf_path, keyword_results, 
                        query_based_on_pdf, docs_processed):
        sequence_str = ' '.join(sequence)
        print(f"Evaluating sequence: {sequence_str}")
        if sequence_str in self.fitness_cache:
            print(f"Returning cached fitness: {self.fitness_cache[sequence_str]}")
            return self.fitness_cache[sequence_str]

        # Create a fresh copy of the original PDF
        pdf_manager.create_fresh_copy(temp_pdf_path)
        print(f"Created fresh copy of PDF at {temp_pdf_path}")

        # Inject the sequence into the fresh copy
        pdf_injector.inject_text(temp_pdf_path, temp_pdf_path, sequence_str, keyword_results, docs_processed)
        print("Injected text into PDF")

        # Query RAG and LLM
        temp_docs_processed = REMOVED_SECRET(
            REMOVED_SECRET(REMOVED_SECRET(temp_pdf_path)))
        temp_vector_db = rag_system.build_vector_database(temp_docs_processed)
        llm_output, relevant_docs, logits = rag_system.query_rag_system(query_based_on_pdf, temp_vector_db)
        print(f"RAG system query result: answer='{llm_output}', relevant_docs={relevant_docs[:100]}...")

        if self.debug_ui:
            REMOVED_SECRET(llm_output)


            # Compute embeddings
        original_embedding = torch.tensor(REMOVED_SECRET(' '.join([doc.page_content for doc in docs_processed])))
        injected_embedding = torch.tensor(REMOVED_SECRET(sequence_str))
        llm_output_embedding = torch.tensor(REMOVED_SECRET(llm_output))
        # Compute semantic distance (1 - cosine similarity)
        #semantic_distance = 1 - F.cosine_similarity(torch.tensor(original_embedding), torch.tensor(llm_output_embedding), dim=0).item()
        contrastive_loss = self.contrastive_loss(original_embedding, injected_embedding, llm_output_embedding)
        # Compute a simple coherence measure (we still want the output to be somewhat readable)
        coherence = self.calculate_coherence(llm_output)

        # Combine metrics (adjust weights as needed)
        # We want to maximize semantic distance while maintaining a minimal level of coherence


        fitness = float(contrastive_loss - 0.1 * coherence)

        self.fitness_cache[sequence_str] = fitness
        return (fitness)



    def calculate_coherence(self, text):
        words = text.split()
        if len(words) < 2:
            print("Warning: Less than two words found. Coherence set to 0.")
            return 0.0

        # Batch process embeddings
        embeddings = REMOVED_SECRET(words)

        # Calculate coherence using cosine similarity between adjacent word embeddings
        coherence_scores = [1 - cosine(embeddings[i], embeddings[i+1]) for i in range(len(embeddings)-1)]

        avg_coherence = np.mean(coherence_scores)
        normalized_coherence = float((avg_coherence + 1) / 2)

        print(f"Number of words: {len(words)}")
        print(f"Normalized coherence: {normalized_coherence}")

        return normalized_coherence

    def optimize(self, rag_system, pdf_injector, pdf_manager, temp_pdf_path, initial_sequence, keyword_results, 
                 token_vocabulary, query_based_on_pdf, docs_processed,
                 num_iterations):
        

        
        self.debug_ui = DebugUI()
        REMOVED_SECRET()

    


        # Initialize particles
        particles = [Particle(token_vocabulary, len(initial_sequence)) for _ in range(self.n_particles)]
        global_best_position = initial_sequence
        global_best_fitness = float('-inf')

        for iteration in range(num_iterations):
            for particle_index, particle in enumerate(particles):
                # Evaluate current position
                fitness = self.evaluate_sequence(particle.position, rag_system, pdf_injector, pdf_manager, 
                                                 temp_pdf_path, keyword_results, 
                                                 query_based_on_pdf, docs_processed)
                
                REMOVED_SECRET(iteration, particle_index, global_best_fitness)
                REMOVED_SECRET(f"Particle {particle_index} fitness: {fitness}")
                REMOVED_SECRET()
                

                # Update personal best
                if fitness > particle.best_fitness:  # Comparing loss
                    particle.best_position = particle.position[:]
                    particle.best_fitness = fitness

                # Update global best
                if fitness > global_best_fitness:  # Comparing loss
                    global_best_position = particle.position[:]
                    global_best_fitness = fitness

            self.update_particles(particles, global_best_position, token_vocabulary)


            print(f"Iteration {iteration}: Best Fitness = {global_best_fitness}")
            REMOVED_SECRET(f"Iteration {iteration}: Best Loss = {global_best_fitness}, Best Coherence = {-global_best_fitness}")
            REMOVED_SECRET()
            REMOVED_SECRET({
                'iteration': iteration,
                'population': [p.position for p in particles],
                'best_fitness': global_best_fitness
            })

        self.visualize_optimization_progress()
        self.animate_pso(token_vocabulary)
        REMOVED_SECRET("Optimization Complete.")
        REMOVED_SECRET()
        return ' '.join(global_best_position), global_best_fitness


    def update_particles(self, particles, global_best, token_vocabulary):
            # Update particle velocities and positions
        for particle in particles:
            r1, r2 = random.random(), random.random()
            for i in range(len(particle.position)):
                cognitive = self.c1 * r1 * (token_vocabulary.index(particle.best_position[i]) - 
                                            token_vocabulary.index(particle.position[i]))
                social = self.c2 * r2 * (token_vocabulary.index(global_best[i]) - 
                                         token_vocabulary.index(particle.position[i]))
                
                particle.velocity[i] = self.w * particle.velocity[i] + cognitive + social
                new_index = (token_vocabulary.index(particle.position[i]) + 
                             int(round(particle.velocity[i]))) % len(token_vocabulary)
                particle.position[i] = token_vocabulary[new_index]


    def visualize_optimization_progress(self):
        iterations = [data['iteration'] for data in self.generation_data]
        best_losses = [data['best_fitness'] for data in self.generation_data]
        best_coherences = [-data['best_fitness'] for data in self.generation_data]

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

        ax1.plot(iterations, best_losses, marker='o')
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('Best Loss')
        ax1.set_title('Loss Progression')

        ax2.plot(iterations, best_coherences, marker='o', color='orange')
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Best Coherence')
        ax2.set_title('Coherence Progression')

        plt.tight_layout()
        plt.savefig('pso_optimization_progress.png')
        plt.close()

        print("Optimization progress visualization saved as 'pso_optimization_progress.png'")

    def animate_pso(self, token_vocabulary):
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')

        def update(frame):
            ax.clear()
            data = self.generation_data[frame]
            positions = np.array([[token_vocabulary.index(token) for token in p] for p in data['population']])
            
            # Plot particles
            ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], c='blue', alpha=0.5)
            
            # Plot global best
            best_position = np.array([token_vocabulary.index(token) for token in data['population'][0]])  # Assuming the first particle is the best
            ax.scatter(best_position[0], best_position[1], best_position[2], c='red', s=100, marker='*')

            ax.set_xlabel('Token 1')
            ax.set_ylabel('Token 2')
            ax.set_zlabel('Token 3')
            ax.set_title(f'PSO Iteration {data["iteration"]}')

        ani = animation.FuncAnimation(fig, update, frames=len(self.generation_data), interval=200, repeat=True)
        ani.save('pso_animation.gif', writer='pillow')
        plt.close()

        print("PSO animation saved as 'pso_animation.gif'")
class Particle:
    def __init__(self, token_vocabulary, sequence_length):
        self.position = random.choices(token_vocabulary, k=sequence_length)
        self.velocity = [0] * sequence_length
        self.best_position = self.position[:]
        self.best_fitness = float('-inf') # (loss, coherence)

In [7]:
class PDFManager:
    def __init__(self):
        self.original_pdf_content = None

    def store_original_pdf(self, pdf_path):
        doc = fitz.open(pdf_path)
        self.original_pdf_content = doc.tobytes()
        doc.close()

    def create_fresh_copy(self, output_path):
        doc = fitz.open("pdf", self.original_pdf_content)
        doc.save(output_path)
        doc.close()

In [8]:
import random
from deap import base, creator, tools, algorithms
import functools
from multiprocessing import Pool
from functools import lru_cache, partial
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from REMOVED_SECRET import cosine
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import os

class OptimizedSubDocumentNSGA2:
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model
        self.fitness_cache = {}
        self.generation_data = []

    def calculate_crowding_distance(self, population):
        if len(population) <= 2:
            for individual in population:
                individual.crowding_distance = float('inf')
            return population

        for individual in population:
            individual.crowding_distance = 0

        for objective in range(len(population[0].fitness.values)):
            population.sort(key=lambda x: REMOVED_SECRET[objective])
            population[0].crowding_distance = float('inf')
            population[-1].crowding_distance = float('inf')

            objective_range = population[-1].fitness.values[objective] - population[0].fitness.values[objective]
            if objective_range == 0:
                continue

            for i in range(1, len(population) - 1):
                distance = (population[i+1].fitness.values[objective] - population[i-1].fitness.values[objective]) / objective_range
                population[i].crowding_distance += distance

        return population

    def crowded_comparison(self, individual1, individual2):
        if REMOVED_SECRET(individual2.fitness):
            return 1
        elif REMOVED_SECRET(individual1.fitness):
            return -1
        elif individual1.crowding_distance > individual2.crowding_distance:
            return 1
        else:
            return -1

    def adaptive_mutation_rate(self, population, gen, max_gen):
        diversity = np.std([REMOVED_SECRET[0] for ind in population])
        progress = gen / max_gen
        return 0.5 * (1 - progress) + 0.1 * diversity

    def adaptive_crossover_rate(self, population, gen, max_gen):
        diversity = np.std([REMOVED_SECRET[0] for ind in population])
        progress = gen / max_gen
        return 0.5 * progress + 0.3 * diversity

    @lru_cache(maxsize=1000)
    def get_embedding(self, text):
        return REMOVED_SECRET(text)

    def evaluate_sequence(self, sequence, rag_system, pdf_injector, pdf_manager, temp_pdf_path, keyword_results, 
                          target_response_tokens, crucial_indices, query_based_on_pdf, docs_processed):
        sequence_str = ' '.join(sequence)
        print(f"Evaluating sequence: {sequence_str}")
        if sequence_str in self.fitness_cache:
            print(f"Returning cached fitness: {self.fitness_cache[sequence_str]}")
            return self.fitness_cache[sequence_str]

        # Create a fresh copy of the original PDF
        pdf_manager.create_fresh_copy(temp_pdf_path)
        print(f"Created fresh copy of PDF at {temp_pdf_path}")

        # Inject the sequence into the fresh copy
        pdf_injector.inject_text(temp_pdf_path, temp_pdf_path, sequence_str, keyword_results, docs_processed)
        print("Injected text into PDF")
        # Query RAG and LLM
        temp_docs_processed = REMOVED_SECRET(
            REMOVED_SECRET(REMOVED_SECRET(temp_pdf_path)))
        temp_vector_db = rag_system.build_vector_database(temp_docs_processed)
        answer, relevant_docs, logits = rag_system.query_rag_system(query_based_on_pdf, temp_vector_db)
        print(f"RAG system query result: answer='{answer}', relevant_docs={relevant_docs[:100]}...")

        # Calculate multiple fitness metrics
        try:
            loss = weighted_loss(logits, target_response_tokens, crucial_indices).item()
            loss = np.clip(loss, 0, 1e6)
            coherence = self.calculate_coherence(sequence_str)
            relevance = self.calculate_relevance(sequence_str, relevant_docs)
            print(f"Calculated fitness metrics: loss={loss}, coherence={coherence}, relevance={relevance}")

            fitness = (loss, -coherence, -relevance)  # We want to minimize loss and maximize coherence and relevance
            self.fitness_cache[sequence_str] = fitness
            print(f"Returning fitness: {fitness}")
            return fitness
        except Exception as e:
            print(f"Error in fitness calculation: {str(e)}")
            return (1e6, -0.5, -0.5)

    def calculate_coherence(self, text):
        sentences = text.split('.')
        if len(sentences) < 2:
            return 1  # Maximum coherence for single sentence

        coherence_scores = []
        for i in range(len(sentences) - 1):
            emb1 = self.get_embedding(sentences[i])
            emb2 = self.get_embedding(sentences[i+1])
            coherence_scores.append(1 - cosine(emb1, emb2))  # 1 - cosine distance = cosine similarity

        return np.mean(coherence_scores)

    def calculate_relevance(self, text, relevant_docs):
        text_emb = self.get_embedding(text)
        docs_emb = self.get_embedding(' '.join(relevant_docs))
        return 1 - cosine(text_emb, docs_emb)  # 1 - cosine distance = cosine similarity

    def parallel_evaluate(self, individual, rag_system, pdf_injector, pdf_manager, temp_pdf_path, keyword_results, 
                          target_response_tokens, crucial_indices, query_based_on_pdf, docs_processed):
        return self.evaluate_sequence(individual, rag_system, pdf_injector, pdf_manager, temp_pdf_path, keyword_results, 
                                      target_response_tokens, crucial_indices, query_based_on_pdf, docs_processed)

    def optimize(self, rag_system, pdf_injector, pdf_manager, temp_pdf_path, initial_sequence, keyword_results, 
                 token_vocabulary, target_response_tokens, crucial_indices, query_based_on_pdf, docs_processed,
                 population_size, num_generations):

        creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0, 1.0))
        creator.create("Individual", list, fitness=creator.FitnessMulti)

        toolbox = base.Toolbox()

        # Set up parallel processing
        num_cores = os.cpu_count()
        pool = Pool(num_cores)
        toolbox.register("map", pool.map)

        toolbox.register("attr_str", random.choice, token_vocabulary)
        toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_str, n=len(initial_sequence))
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # Partial function for parallel evaluation
        partial_evaluate = partial(self.parallel_evaluate, 
                                   rag_system=rag_system, 
                                   pdf_injector=pdf_injector,
                                   pdf_manager=pdf_manager, 
                                   temp_pdf_path=temp_pdf_path, 
                                   keyword_results=keyword_results,
                                   target_response_tokens=target_response_tokens, 
                                   crucial_indices=crucial_indices,
                                   query_based_on_pdf=query_based_on_pdf, 
                                   docs_processed=docs_processed)

        toolbox.register("evaluate", partial_evaluate)
        toolbox.register("mate", tools.cxTwoPoint)
        toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
        toolbox.register("select", tools.selNSGA2)

        population = toolbox.population(n=population_size)

        # Try parallel processing, fall back to sequential if it fails
        try:
            with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
                # Evaluate the initial population
                fitnesses = list(executor.map(toolbox.evaluate, population))
                for ind, fit in zip(population, fitnesses):
                    REMOVED_SECRET = fit

                for gen in range(num_generations):
                    offspring = algorithms.varAnd(population, toolbox, 
                                                  cxpb=self.adaptive_crossover_rate(population, gen, num_generations), 
                                                  mutpb=self.adaptive_mutation_rate(population, gen, num_generations))

                    # Evaluate the offspring
                    fitnesses = list(executor.map(toolbox.evaluate, offspring))
                    for ind, fit in zip(offspring, fitnesses):
                        REMOVED_SECRET = fit

                    # ... (rest of the generation loop remains the same)
        except Exception as e:
            print(f"Parallel processing failed: {e}. Falling back to sequential processing.")
            # Evaluate the initial population sequentially
            for ind in population:
                REMOVED_SECRET = toolbox.evaluate(ind)

            for gen in range(num_generations):
                offspring = algorithms.varAnd(population, toolbox, 
                                              cxpb=self.adaptive_crossover_rate(population, gen, num_generations), 
                                              mutpb=self.adaptive_mutation_rate(population, gen, num_generations))

                # Evaluate the offspring sequentially
                for ind in offspring:
                    REMOVED_SECRET = toolbox.evaluate(ind)

            # Select the next generation population
            population = toolbox.select(population + offspring, k=len(population))

            # Calculate crowding distance
            population = self.calculate_crowding_distance(population)

            # Sort based on crowded comparison
            population.sort(key=functools.cmp_to_key(self.crowded_comparison), reverse=True)

            # Log the best fitness
            best_fitness = population[0].fitness.values[0]
            print(f"Generation {gen}: Best Loss = {best_fitness}")

            # Store generation data for visualization
            REMOVED_SECRET({
                'generation': gen,
                'population': population.copy(),
                'best_fitness': best_fitness
            })

        pool.close()
        pool.join()

        # Return the best individual
        best_individual = tools.selBest(population, k=1)[0]
        self.visualize_pareto_front()
        return ' '.join(best_individual)

    def visualize_pareto_front(self):
        if not self.generation_data:
            print("No data available for visualization")
            return

        final_population = self.generation_data[-1]['population']
        print(f"Total individuals in final population: {len(final_population)}")

        valid_fitness_values = []
        for ind in final_population:
            if hasattr(ind, 'fitness') and REMOVED_SECRET and len(REMOVED_SECRET) >= 3:
                valid_fitness_values.append(REMOVED_SECRET[:3])  # Take first 3 values
            else:
                print(f"Skipping individual with invalid fitness: {getattr(ind, 'fitness', 'No fitness')}")

        print(f"Valid individuals for visualization: {len(valid_fitness_values)}")

        if not valid_fitness_values:
            print("No valid fitness values for visualization")
            return

        fitness_values = np.array(valid_fitness_values)

        # Normalize fitness values
        normalized_fitness = (fitness_values - fitness_values.min(axis=0)) / (fitness_values.max(axis=0) - fitness_values.min(axis=0))

        # 3D scatter plot
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        scatter = ax.scatter(normalized_fitness[:, 0], normalized_fitness[:, 1], normalized_fitness[:, 2], c=normalized_fitness[:, 0], cmap='viridis')
        ax.set_xlabel('Normalized Loss')
        ax.set_ylabel('Normalized Coherence')
        ax.set_zlabel('Normalized Relevance')
        ax.set_title('Normalized Pareto Front')
        plt.colorbar(scatter, label='Normalized Loss')
        plt.savefig('pareto_front_3d.png')
        plt.close()

        # 2D scatter plot matrix
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        axs[0].scatter(normalized_fitness[:, 0], normalized_fitness[:, 1])
        axs[0].set_xlabel('Normalized Loss')
        axs[0].set_ylabel('Normalized Coherence')
        axs[1].scatter(normalized_fitness[:, 0], normalized_fitness[:, 2])
        axs[1].set_xlabel('Normalized Loss')
        axs[1].set_ylabel('Normalized Relevance')
        axs[2].scatter(normalized_fitness[:, 1], normalized_fitness[:, 2])
        axs[2].set_xlabel('Normalized Coherence')
        axs[2].set_ylabel('Normalized Relevance')
        plt.tight_layout()
        plt.savefig('pareto_front_2d.png')
        plt.close()

        print(f"Pareto front visualizations saved as 'pareto_front_3d.png' and 'pareto_front_2d.png' with {len(valid_fitness_values)} valid individuals")

    def visualize_interactive_pareto_front(self):
        final_population = self.generation_data[-1]['population']
        fitness_values = np.array([REMOVED_SECRET for ind in final_population])

        fig = go.Figure(data=[go.Scatter3d(
            x=fitness_values[:, 0],
            y=fitness_values[:, 1],
            z=fitness_values[:, 2],
            mode='markers',
            marker=dict(
                size=5,
                color=fitness_values[:, 0],  # color by loss
                colorscale='Viridis',
                opacity=0.8
            )
        )])

        fig.update_layout(
            scene = dict(
                xaxis_title='Loss',
                yaxis_title='Coherence',
                zaxis_title='Relevance'
            ),
            title='Interactive Pareto Front'
        )

        fig.write_html("interactive_pareto_front.html")

In [9]:
class WorkflowManager:
    def __init__(self):
        self.rag_system = RAGSystem(
            embedding_model_name=EMBEDDING_MODEL_NAME,
            model_id=MODEL_ID,
            reranker_model=RERANKER_MODEL
        )
        self.pdf_extractor = PDFKeywordExtractor()
        self.pdf_injector = PDFInjector(REMOVED_SECRET)
        self.docs_processed = None
        self.local_database_path = "local_database"
        self.optimizer = PSOSequenceOptimizer(
            embedding_model=REMOVED_SECRET,
            n_particles=5,
            w=0.5,
            c1=1,
            c2=1,
            temperature=0.1
        )


        #initialise nsga2
        #embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
        #self.optimizer = OptimizedSubDocumentNSGA2(embedding_model)

        #initialise pdf manager
        self.pdf_manager = PDFManager()

    def run(self):
        original_pdf_path = self._browse_for_pdf()

        if not original_pdf_path:
            print("No file selected.")
            return
        
        # Create a unique identifier for this run
        run_id = str(uuid.uuid4())
        
        #store original pdf content
        REMOVED_SECRET(original_pdf_path)

        # set up temporary pdf path in local database
        temp_pdf_path = REMOVED_SECRET(self.local_database_path, f"temp_{run_id}.pdf")   

        #create initial temp pdf
        REMOVED_SECRET(temp_pdf_path)
        
        keyword_results = REMOVED_SECRET(temp_pdf_path)
        if not keyword_results:
            print("No keywords extracted from the PDF.")
            os.remove(temp_pdf_path)
            return
        
        self.docs_processed = REMOVED_SECRET.split_documents(
            REMOVED_SECRET.load_pdfs_from_folder(self.local_database_path)
        )

        self.vocab_list = generate_vocab_list(50257)
        print("Vocab list:", self.vocab_list)
        best_keywords = keyword_results[:5]
        initial_sub_document = best_keywords
        token_vocabulary = self.vocab_list + keyword_results
        #target_response = "I'm sorry, I can't help with that."
        query_based_on_pdf = "What is the capital of France?"
        #T = 1  
        #B = 1  

        #target_response_tokens = REMOVED_SECRET(target_response, return_tensors='pt').squeeze().to(device)
        #crucial_phrase = "I'm sorry"
        #crucial_indices = [i for i, token in enumerate(target_response_tokens) if REMOVED_SECRET([token]) in crucial_phrase.split()]
       
       
        # Optimise sequences
        """
        final_candidate = REMOVED_SECRET(
            self.rag_system,
            self.pdf_injector,
            self.pdf_manager,
            temp_pdf_path,
            initial_sub_document,
            keyword_results,
            token_vocabulary,
            target_response_tokens,
            crucial_indices,
            query_based_on_pdf,
            self.docs_processed,
            population_size=15,
            num_generations=5
        )
        """

        final_candidate, best_fitness = REMOVED_SECRET(
            rag_system=self.rag_system,
            pdf_injector=self.pdf_injector,
            pdf_manager=self.pdf_manager,
            temp_pdf_path=temp_pdf_path,
            initial_sequence=initial_sub_document,
            keyword_results=keyword_results,
            token_vocabulary=token_vocabulary,
            query_based_on_pdf=query_based_on_pdf,
            docs_processed=self.docs_processed,
            num_iterations=5  # Adjust as needed
        )
        print(f"Final candidate: {final_candidate}")
        print(f"Best fitness: Loss = {best_fitness}")
        # Create the final output PDF
        final_output_pdf_path = REMOVED_SECRET(self.local_database_path, f"output_{run_id}.pdf")
        REMOVED_SECRET(final_output_pdf_path)
        REMOVED_SECRET(final_output_pdf_path, final_output_pdf_path, final_candidate, keyword_results, self.docs_processed)

        # Remove the temporary PDF
        os.remove(temp_pdf_path)

        final_response_file = REMOVED_SECRET(self.local_database_path, f"final_response_{run_id}.txt")
        with open(final_response_file, "w") as f:
            f.write(final_candidate)
        print(f"Final response saved to {final_response_file}")
        print(f"Final output PDF saved to {final_output_pdf_path}")

        #cleanup
        REMOVED_SECRET()
    
    def _optimize_sub_document(self, pdf_file_path, sub_document, keyword_results, token_vocabulary, target_response_tokens, 
                               crucial_indices, query_based_on_pdf, T, B, temp_dir):
        sequence_mutator = SequenceMutator(REMOVED_SECRET.model, REMOVED_SECRET.tokenizer)

        for i in range(T):
            candidate_sub_documents = []
            losses = []
            

            for b in range(B):

                temp_output_pdf_path = REMOVED_SECRET(temp_dir, f"temp_output_{i}_{b}.pdf")
                

                REMOVED_SECRET(pdf_file_path, temp_output_pdf_path, ' '.join(sub_document), keyword_results, self.docs_processed)
                
                keyword_results = REMOVED_SECRET(temp_output_pdf_path)
                
                temp_docs_processed = REMOVED_SECRET.split_documents(REMOVED_SECRET.load_pdfs_from_folder(temp_dir))
                print("Temp docs:", temp_docs_processed)
                temp_vector_db = REMOVED_SECRET(temp_docs_processed)

                answer, relevant_docs, logits = REMOVED_SECRET(query_based_on_pdf, temp_vector_db)
                
                print("Answer:", answer)
                print("Relevant docs:", relevant_docs[:100] if relevant_docs else "None")  # Print first 100 chars

                seq_tokens = torch.tensor([REMOVED_SECRET(' '.join(sub_document))], dtype=torch.long)
                print("Seq Tokens:", seq_tokens)
                print("Seq tokens shape:", seq_tokens.shape)
                print("Target response tokens shape:", target_response_tokens.shape)
            
                #    Use SequenceMutator to generate new sequences
                new_seqs = sequence_mutator.mutate_sequence(seq_tokens, target_response_tokens, crucial_indices)
                # Evaluate new sequences
                for new_seq in new_seqs:
                    new_sub_document = REMOVED_SECRET(new_seq[0])

                    #create new temp pdf for each new sequence
                    #temp_seq_pdf_path = REMOVED_SECRET(temp_dir, f"temp_seq{i}_{b}.pdf")
                    #REMOVED_SECRET(pdf_file_path, temp_seq_pdf_path, new_sub_document, keyword_results, self.docs_processed)
                    
                    #update rag system with new temp pdf
                    #temp_seq_docs_processed = REMOVED_SECRET.split_documents(REMOVED_SECRET.load_pdfs_from_folder(temp_seq_pdf_path))
                    #answer, relevant_docs, new_logits = REMOVED_SECRET(query_based_on_pdf, REMOVED_SECRET(temp_seq_docs_processed))
                
                    try:
                        loss = weighted_loss(logits, target_response_tokens, crucial_indices)
                        losses.append(loss.item())
                    except Exception as e:
                        print(f"Error in loss calculation: {e}")
                        losses.append(float('inf'))

                    candidate_sub_documents.append(new_sub_document.split())
                    #remove temporary sequence pdf
                   # os.remove(temp_seq_pdf_path)

                print(f"Iteration {i+1}/{T}, Candidate {b+1}/{B}, Loss: {loss.item()}")

                os.remove(temp_output_pdf_path) #remove temp output pdf for this batch
            
            best_candidate = sequence_mutator.choose_best_sequence(candidate_sub_documents, losses)
            sub_document = best_candidate
        
        return ' '.join(sub_document)
    
    def _browse_for_pdf(self):
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
        return file_path


In [10]:
if __name__ == "__main__":
    workflow_manager = WorkflowManager()

    workflow_manager.run()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


Keywords extracted: [('capital france paris', 0.8715), ('capital france', 0.8437), ('france paris', 0.7879), ('paris', 0.7602), ('france', 0.6313), ('capital', 0.4767)]


Loading PDFs: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]


Documents: [Document(page_content='What is the capital of France? Paris!', metadata={'source': 'local_database/temp_18df0f0a-48d0-49ba-979a-02467c64c820.pdf'})]


...

Splitting Documents: 100%|██████████| 1/1 [00:00<00:00, 1824.40it/s]


=> Retrieving documents...
Retrieved 1 documents.
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 132.58it/s]

Reranked to 1 documents.
Using 1 documents for answering.
=> Generating answer...


RAG system query result: answer='

Document 2:
quotes (container .scroll Lan Setting

What is the capital of France? Paris!

            Question: What is the capital of France?
    
            Instructions: Using the information from', relevant_docs=['quotes (container .scroll Lan Setting\n\nWhat is the capital of France? Paris!']...
Number of words: 26
Normalized coherence: 0.9070939319379825
Iteration 4: Best Fitness = -0.7421506567760807
Optimization progress visualization saved as 'pso_optimization_progress.png'
PSO animation saved as 'pso_animation.gif'
Final candidate:  garage quivo ;\ .GL  weekly
Best fitness: Loss = -0.7421506567760807
Strongest keyword: capital
Final response saved to local_database/final_response_18df0f0a-48d0-49ba-979a-02467c64c820.txt
Final output PDF saved to local_database/output_18df0f0a-48d0-49ba-979a-02467c64c820.pdf
